In [2]:
import numpy as np
import pandas as pd

In [3]:
df=pd.read_csv("covid_large_dataset_50000_rows.csv")

In [4]:
df.isna().sum()

date                       5000
location                   5000
total_cases                5000
new_cases                  5000
total_deaths               5000
new_deaths                 5000
people_vaccinated          5000
people_fully_vaccinated    5000
new_tests                  5000
stringency_index           5000
population                 5000
population_density         5000
hospital_patients          5000
icu_patients               5000
reproduction_rate          5000
dtype: int64

In [5]:
df

,date,location,total_cases,new_cases,total_deaths,new_deaths,people_vaccinated,people_fully_vaccinated,new_tests,stringency_index,population,population_density,hospital_patients,icu_patients,reproduction_rate
0,2020-01-01,Germany,1667072.0,11044.0,75527.0,587.0,22790780.0,NaN,90608.0,74.445725,3.300000e+08,51.006798,2607.0,383.0,0.747185
1,2020-01-02,UK,NaN,8285.0,94054.0,472.0,27983062.0,24392755.0,77276.0,NaN,8.300000e+07,834.486127,3803.0,417.0,1.582235
2,2020-01-03,Russia,1236229.0,19757.0,61740.0,576.0,27818173.0,14663185.0,90997.0,NaN,1.400000e+09,NaN,2499.0,419.0,0.606501
3,2020-01-04,Germany,1501600.0,12176.0,60371.0,549.0,34525525.0,23930158.0,62069.0,60.594745,6.000000e+07,458.220451,2669.0,382.0,1.022295
4,2020-01-05,Brazil,1214245.0,12592.0,48861.0,445.0,29163824.0,21244642.0,112210.0,32.089016,2.120000e+08,417.887168,2795.0,489.0,1.844666
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49995,2156-11-18,France,1212152.0,19635.0,92010.0,534.0,29643245.0,29230722.0,81486.0,35.528719,2.120000e+08,830.013515,2387.0,278.0,0.796539
49996,2156-11-19,France,1634220.0,16352.0,39559.0,386.0,21322261.0,25538991.0,68941.0,76.759229,2.120000e+08,564.977199,3465.0,NaN,1.691444
49997,2156-11-20,Brazil,NaN,21227.0,NaN,372.0,32895876.0,15357062.0,53526.0,NaN,1.450000e+08,170.730685,2958.0,174.0,0.791688
49998,2156-11-21,USA,1930692.0,15117.0,109126.0,363.0,30554772.0,26767251.0,97719.0,21.623927,1.400000e+09,1210.140684,NaN,397.0,1.011190


In [6]:
df.dtypes

date                        object
location                    object
total_cases                float64
new_cases                  float64
total_deaths               float64
new_deaths                 float64
people_vaccinated          float64
people_fully_vaccinated    float64
new_tests                  float64
stringency_index           float64
population                 float64
population_density         float64
hospital_patients          float64
icu_patients               float64
reproduction_rate          float64
dtype: object

In [7]:
if "new_cases" in df.columns:

    df["new_cases_avg"] = df["new_cases"].rolling(7,min_periods=1).mean()

In [8]:
numerical_columns= df.select_dtypes(include=["int64","float64"]).columns.to_list()
categorical_columns=df.select_dtypes(include=["object"]).columns.to_list()

In [9]:
from sklearn.impute import SimpleImputer

li=SimpleImputer(strategy="median")

df[numerical_columns]= li.fit_transform(df[numerical_columns])

In [10]:
le=SimpleImputer(strategy="most_frequent")

df[categorical_columns] = le.fit_transform(df[categorical_columns])

In [11]:
from sklearn.preprocessing import LabelEncoder

label=LabelEncoder()

df["date"] = label.fit_transform(df["date"])

df["location"] = label.fit_transform(df["location"])

In [12]:
X = df.drop(["new_cases"],axis=1)

y=df["new_cases_avg"]

In [13]:
from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2)

In [14]:
from sklearn.preprocessing import StandardScaler

scaler=StandardScaler()

X_train= scaler.fit_transform(X_train)

X_test = scaler.transform(X_test)

In [15]:
from sklearn.ensemble import RandomForestRegressor

rf=RandomForestRegressor(n_estimators=200,max_depth=15,random_state=42)

rf.fit(X_train,y_train)

y_pred= rf.predict(X_test)


In [16]:
from sklearn.metrics import r2_score

r2_score(y_pred,y_test)

0.9999979212644527

In [17]:
from sklearn.metrics import mean_absolute_percentage_error

mean_absolute_percentage_error(y_pred,y_test)

1.7695686828394562e-05

In [18]:
df["new_cases_avg"]

0        11044.000000
1         9664.500000
2        13028.666667
3        12815.500000
4        12770.800000
             ...     
49995    16367.285714
49996    16653.714286
49997    17246.000000
49998    17092.857143
49999    16587.857143
Name: new_cases_avg, Length: 50000, dtype: float64